In [1]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [2]:
##dataframe = sqlContext.read.format('csv').options(header='true', inferScheme='true').load('C:/Users/jazmi/OneDrive/Documents/DataMate/db/all.csv')
df = spark.table("all_csv")

df.columns

Out[ 2 ]: ['_c0', 'username', 'Gender', 'Education', 'Ethnicity', 'about_me_text']

In [3]:
df = df.select('username', 'about_me_text').na.drop()
df.show()

+--------------+--------------------+
 username| about_me_text|
+--------------+--------------------+
ShortyFire1984|....................|
 ROXYCHIC549|'" I am not an up...|
 tequilagirl69|Laid back lovable...|
atxredneckgirl|Tx country music ...|
 mel12th|I am here to look...|
 Andi010274|Can't get into my...|
 nodutch|Music business go...|
 lauranmr|Of all the dating...|
 smbd75|Likes: fun, adven...|
 candipie1989|Single black fema...|
 tif1985|I'm on here just ...|
 De1964|Please local only...|
 Trini1971|I am a Mom to a 1...|
 FiaFia1213|I like to think I...|
 SweetJojos| Up for anything|
 Tawdra83|I’m a artist. I c...|
 LadyTexas254|not looking for a...|
 Vigilia79|Hi there! I'm Chr...|
 ayoung292|Music is life and...|
 Foreverfungrl| Ask me Are|
+--------------+--------------------+
only showing top 20 rows

In [4]:
# Tokenize data
tokened_df = Tokenizer(inputCol="about_me_text", outputCol="words")
tt_df = tokened_df.transform(df)
tt_df.show()


+--------------+--------------------+--------------------+
 username| about_me_text| words|
+--------------+--------------------+--------------------+
ShortyFire1984|....................|[...................|
 ROXYCHIC549|'" I am not an up...|['", i, am, not, ...|
 tequilagirl69|Laid back lovable...|[laid, back, lova...|
atxredneckgirl|Tx country music ...|[tx, country, mus...|
 mel12th|I am here to look...|[i, am, here, to,...|
 Andi010274|Can't get into my...|[can't, get, into...|
 nodutch|Music business go...|[music, business,...|
 lauranmr|Of all the dating...|[of, all, the, da...|
 smbd75|Likes: fun, adven...|[likes:, fun,, ad...|
 candipie1989|Single black fema...|[single, black, f...|
 tif1985|I'm on here just ...|[i'm, on, here, j...|
 De1964|Please local only...|[please, local, o...|
 Trini1971|I am a Mom to a 1...|[i, am, a, mom, t...|
 FiaFia1213|I like to think I...|[i, like, to, thi...|
 SweetJojos| Up for anything| [up, for, anything]|
 Tawdra83|I’m a artist. I c...|[i’m, a, artist.,...|
 LadyTexas254|not looking for a...|[not, looking, fo...|
 Vigilia79|Hi there! I'm Chr...|[hi, there!, i'm,...|
 ayoung292|Music is life and...|[music, is, life,...|
 Foreverfungrl| Ask me Are| [ask, me, are]|
+--------------+--------------------+--------------------+
only showing top 20 rows

In [5]:
# Run the hashing term frequency
hashing = HashingTF(inputCol='words', outputCol="hashedValues", numFeatures=20)

# Transform into a DF
hashed_df = hashing.transform(tt_df)

In [6]:
hashed_df.show()

+--------------+--------------------+--------------------+--------------------+
 username| about_me_text| words| hashedValues|
+--------------+--------------------+--------------------+--------------------+
ShortyFire1984|....................|[...................| (20,[5],[1.0])|
 ROXYCHIC549|'" I am not an up...|['", i, am, not, ...|(20,[0,1,2,3,4,5,...|
 tequilagirl69|Laid back lovable...|[laid, back, lova...|(20,[0,1,2,4,5,6,...|
atxredneckgirl|Tx country music ...|[tx, country, mus...|(20,[0,1,2,3,4,5,...|
 mel12th|I am here to look...|[i, am, here, to,...|(20,[0,1,2,3,4,5,...|
 Andi010274|Can't get into my...|[can't, get, into...|(20,[1,8,9,11,13,...|
 nodutch|Music business go...|[music, business,...|(20,[0,2,3,10,13,...|
 lauranmr|Of all the dating...|[of, all, the, da...|(20,[0,1,2,3,4,5,...|
 smbd75|Likes: fun, adven...|[likes:, fun,, ad...|(20,[0,7,9,13,16,...|
 candipie1989|Single black fema...|[single, black, f...|(20,[0,1,2,3,4,5,...|
 tif1985|I'm on here just ...|[i'm, on, here, j...|(20,[0,1,2,3,4,5,...|
 De1964|Please local only...|[please, local, o...|(20,[0,1,2,3,4,5,...|
 Trini1971|I am a Mom to a 1...|[i, am, a, mom, t...|(20,[0,1,3,4,5,6,...|
 FiaFia1213|I like to think I...|[i, like, to, thi...|(20,[0,2,3,4,5,6,...|
 SweetJojos| Up for anything| [up, for, anything]|(20,[8,14,16],[1....|
 Tawdra83|I’m a artist. I c...|[i’m, a, artist.,...|(20,[0,1,2,3,4,5,...|
 LadyTexas254|not looking for a...|[not, looking, fo...|(20,[0,1,2,3,4,5,...|
 Vigilia79|Hi there! I'm Chr...|[hi, there!, i'm,...|(20,[0,1,2,3,4,5,...|
 ayoung292|Music is life and...|[music, is, life,...|(20,[1,3,4,8,9,11...|
 Foreverfungrl| Ask me Are| [ask, me, are]|(20,[11,14,18],[1...|
+--------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [7]:
# Fit the IDF on the data set 
idf = IDF(inputCol="hashedValues", outputCol="features")
idfModel = idf.fit(hashed_df)
rescaledData = idfModel.transform(hashed_df)

In [8]:
# Display the DataFrame
rescaledData.select("words", "features").show()

+--------------------+--------------------+
 words| features|
+--------------------+--------------------+
[...................|(20,[5],[0.247209...|
['", i, am, not, ...|(20,[0,1,2,3,4,5,...|
[laid, back, lova...|(20,[0,1,2,4,5,6,...|
[tx, country, mus...|(20,[0,1,2,3,4,5,...|
[i, am, here, to,...|(20,[0,1,2,3,4,5,...|
[can't, get, into...|(20,[1,8,9,11,13,...|
[music, business,...|(20,[0,2,3,10,13,...|
[of, all, the, da...|(20,[0,1,2,3,4,5,...|
[likes:, fun,, ad...|(20,[0,7,9,13,16,...|
[single, black, f...|(20,[0,1,2,3,4,5,...|
[i'm, on, here, j...|(20,[0,1,2,3,4,5,...|
[please, local, o...|(20,[0,1,2,3,4,5,...|
[i, am, a, mom, t...|(20,[0,1,3,4,5,6,...|
[i, like, to, thi...|(20,[0,2,3,4,5,6,...|
 [up, for, anything]|(20,[8,14,16],[0....|
[i’m, a, artist.,...|(20,[0,1,2,3,4,5,...|
[not, looking, fo...|(20,[0,1,2,3,4,5,...|
[hi, there!, i'm,...|(20,[0,1,2,3,4,5,...|
[music, is, life,...|(20,[1,3,4,8,9,11...|
 [ask, me, are]|(20,[11,14,18],[0...|
+--------------------+--------------------+
only showing top 20 rows